# Implemention of the paper "Retrieve, Caption, Generate: Visual Grounding for Enhancing Commonsense in Text Generation Models" by Mitamura et al.
# --By Ritirupa Dey

## The paper proposes that incorporating visual grounding improves the performance of Text Generation Models in the task of Constrained Natural Language Generation and has also introduced a new model - VisCTG to demonstrate it.

## I have tried implementing the paper on a smaller dataset(due to resource limitations) and verified the same.

#Installing necessary packages and libraries

In [1]:
!pip install opendatasets
!pip install simple_image_download
!pip install datasets transformers[sentencepiece]
!pip install sentencepiece

INFO: pip is looking at multiple versions of simple-image-download to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from simple_image_download import simple_image_download as simp
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor
from tqdm import tqdm
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
tqdm.pandas()
import opendatasets as od
import ast
import pandas
import urllib.request
od.download("https://www.kaggle.com/datasets/thedevastator/commongen-a-benchmark-dataset-for-generative-com")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ritirupadey
Your Kaggle Key: ··········


100%|██████████| 1.86M/1.86M [00:01<00:00, 1.78MB/s]

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


#Downloading the datasets

In [12]:
train_df_ori=pd.read_csv('/content/commongen-a-benchmark-dataset-for-generative-com/train.csv')
val_df_ori=pd.read_csv('/content/commongen-a-benchmark-dataset-for-generative-com/validation.csv')
test_df_ori=pd.read_csv('/content/commongen-a-benchmark-dataset-for-generative-com/test.csv')
train_df_ori

,concept_set_idx,concepts,target
0,0,['ski' 'mountain' 'skier'],Skier skis down the mountain
1,0,['ski' 'mountain' 'skier'],A skier is skiing down a mountain.
2,0,['ski' 'mountain' 'skier'],Three skiers are skiing on a snowy mountain.
3,1,['wag' 'tail' 'dog'],The dog is wagging his tail.
4,1,['wag' 'tail' 'dog'],A dog wags his tail at the boy.
...,...,...,...
67384,32646,['aspect' 'worship' 'temple' 'feature' 'type'],type of place of worship of artwork featuring ...
67385,32647,['feature' 'aspect' 'temple' 'place' 'artwork'],type of place of worship of artwork featuring ...
67386,32648,['aspect' 'feature' 'place' 'artwork' 'type'],type of place of worship of artwork featuring ...
67387,32649,['type' 'temple' 'aspect' 'artwork' 'feature'],type of place of worship of artwork featuring ...


# Preprocessing the train and test data

Splitting the data into train and test sets, keeping the distribution of the concepts intact in each, as mentioned in the paper. Train set contains 500 concepts of length 3, 200 concepts of length 4 and 200 concepts of length 5, while the test set contains 100 concepts of length 4 and 5 each.

In [13]:
train_df_ori['concepts_processed']=train_df_ori['concepts'].apply(lambda x:[y.strip("'") for y in x[1:-1].split(" ")])
grouped = train_df_ori.groupby('concepts').first()
boolean_mask1 = grouped['concepts_processed'].map(lambda x: len(x) == 3)
train_df_filtered1 =grouped.loc[boolean_mask1][:500]
#test_df_filtered1=test_df.loc[boolean_mask1][500:700]
boolean_mask2 = grouped['concepts_processed'].map(lambda x: len(x) == 4)
train_df_filtered2 =grouped.loc[boolean_mask2][:200]
test_df_filtered2=grouped.loc[boolean_mask2][200:300]
boolean_mask3 = grouped['concepts_processed'].map(lambda x: len(x) == 5)
train_df_filtered3 =grouped.loc[boolean_mask3][:200]
test_df_filtered3=grouped.loc[boolean_mask3][200:300]
X_train = pd.concat([train_df_filtered1, train_df_filtered2, train_df_filtered3], axis=0)
X_test=pd.concat([test_df_filtered2,test_df_filtered3])X_train['concepts_new']=X_train['concepts_processed'].apply(lambda x: "+".join(x))
X_test['concepts_new']=X_test['concepts_processed'].apply(lambda x: "+".join(x))
X_train=X_train.reset_index()
X_test=X_test.reset_index()

In [21]:
X_test

,concepts,concept_set_idx,target,concepts_processed,concepts_new
0,['basket' 'fruit' 'orange' 'include'],25371,Multiple baskets filled with fruit including o...,"[basket, fruit, orange, include]",basket+fruit+orange+include
1,['basket' 'fry' 'topping' 'hotdog'],27622,A hotdog with toppings next to a basket of sea...,"[basket, fry, topping, hotdog]",basket+fry+topping+hotdog
2,['bassist' 'concert' 'live' 'perform'],27974,bassist of filter performs live in concert,"[bassist, concert, live, perform]",bassist+concert+live+perform
3,['bathroom' 'cabinet' 'window' 'sink'],25587,"A bathroom with a sink, counter, cabinets and ...","[bathroom, cabinet, window, sink]",bathroom+cabinet+window+sink
4,['bathroom' 'checker' 'sink' 'floor'],25400,a bathroom with a sink and a checkered floor,"[bathroom, checker, sink, floor]",bathroom+checker+sink+floor
...,...,...,...,...,...
195,['building' 'city' 'horizon' 'illuminate' 'road'],30846,asphalt road and city with illuminated buildin...,"[building, city, horizon, illuminate, road]",building+city+horizon+illuminate+road
196,['building' 'cloud' 'reflect' 'sky' 'facade'],29503,sky and clouds reflecting over the facades of ...,"[building, cloud, reflect, sky, facade]",building+cloud+reflect+sky+facade
197,['building' 'colour' 'detail' 'add' 'attention'],32376,attention to details and vibrant colours add t...,"[building, colour, detail, add, attention]",building+colour+detail+add+attention
198,['building' 'detail' 'attention' 'appeal' 'add'],32379,attention to details and vibrant colours add t...,"[building, detail, attention, appeal, add]",building+detail+attention+appeal+add


In [15]:
X_train

,concepts,concept_set_idx,target,concepts_processed,concepts_new
0,['abacus' 'help' 'learn'],20011,these young girls learn the art of calculating...,"[abacus, help, learn]",abacus+help+learn
1,['abandon' 'bank' 'boat'],3747,an abandoned boat lies on the bank .,"[abandon, bank, boat]",abandon+bank+boat
2,['abandon' 'building' 'prairie'],14195,windmill and buildings of an old abandoned hom...,"[abandon, building, prairie]",abandon+building+prairie
3,['abandon' 'building' 'town'],3509,a derelict building in the abandoned town,"[abandon, building, town]",abandon+building+town
4,['abandon' 'chair' 'asylum'],16115,chairs in an abandoned insane asylum,"[abandon, chair, asylum]",abandon+chair+asylum
...,...,...,...,...,...
895,['block' 'flat' 'sky' 'post' 'corner'],31802,a block of flats viewed up towards the patchy ...,"[block, flat, sky, post, corner]",block+flat+sky+post+corner
896,['block' 'flower' 'rear' 'grow' 'garden'],31463,macro image of flowers growing in the ornament...,"[block, flower, rear, grow, garden]",block+flower+rear+grow+garden
897,['block' 'sun' 'frond' 'tree' 'cloud'],31830,tropical storm clouds block out the sun as win...,"[block, sun, frond, tree, cloud]",block+sun+frond+tree+cloud
898,['block' 'tree' 'cloud' 'wind' 'sun'],31834,tropical storm clouds block out the sun as win...,"[block, tree, cloud, wind, sun]",block+tree+cloud+wind+sun


# Retrieve!
The top 20 urls are fetched corresponding to each concept set in the train and test images. However, they often contain images which are thumbnails or icons and have no visual interpretation. Also some urls are private and retrieving images from those sites are forbidden and throws up error. So from the 20 urls, 2 images have been retrieved corresponding to each concept set.

In [ ]:
import pickle
p1=[]
checkpoint_interval=100
images_dir='/content/drive/MyDrive/Retrieve_caption_generate/test_images'
import urllib
count=0
for i in range(len(X_test)):
    l = []
    s = "+".join(X_test['concepts_processed'][i])
    responses = simp.simple_image_download
    l1 = responses().urls(s, 20)

    for x in l1:
        if x.find("icon") == -1 and x.find("thumbnail") == -1:
            l.append(x)

    p = []
    for j in l[-1:-3:-1]:
        try:
            img = urllib.request.urlretrieve(j, images_dir + '/' + ("+".join(X_test['concepts_processed'][i])) + "_"+str(count) + ".png")
            p.append(img[0])
            count += 1
        except:
            continue

    X_test.loc[i, 'Image_path'] = str(p)
    p1.append(p)
    print(count)

    # Create a checkpoint every checkpoint_interval iterations
    if i % checkpoint_interval == 0:
        with open('/content/checkpoint_' + str(i) + '.pkl', 'wb') as f:
            pickle.dump([X_test, p1, count,i], f)




In [21]:
import pickle
iteration_number=200
with open('/content/checkpoint_'+str(iteration_number)+'.pkl', 'rb') as f:
    checkpoint_data = pickle.load(f)
X_train = checkpoint_data[0]
p1 = checkpoint_data[1]
count = checkpoint_data[2]
y=checkpoint_data[3]

In [ ]:
for i in range(count,len(X_test)):
    l = []
    s = "+".join(X_test['concepts_processed'][i])
    responses = simp.simple_image_download
    l1 = responses().urls(s, 20)

    for x in l1:
        if x.find("icon") == -1 and x.find("thumbnail") == -1:
            l.append(x)

    p = []
    for j in l[-1:-3:-1]:
        try:
            img = urllib.request.urlretrieve(j, images_dir + '/' + ("+".join(X_test['concepts_processed'][i])) + "_"+str(count) + ".png")
            p.append(img[0])
            count += 1
        except:
            continue

    X_test.loc[i, 'Image_path'] = str(p)
    p1.append(p)
    print(count)

    # Create a checkpoint every checkpoint_interval iterations
    if i % checkpoint_interval == 0:
        with open('/content/checkpoint_' + str(i) + '.pkl', 'wb') as f:
            pickle.dump([X_test, p1, count,i], f)

In [ ]:
import pickle
p1=[]
checkpoint_interval=100
images_dir='/content/drive/MyDrive/Retrieve_caption_generate/train_images'
import urllib
count=0
for i in range(len(X_train)):
    l = []
    s = "+".join(X_train['concepts_processed'][i])
    responses = simp.simple_image_download
    l1 = responses().urls(s, 20)

    for x in l1:
        if x.find("icon") == -1 and x.find("thumbnail") == -1:
            l.append(x)

    p = []
    for j in l[-1:-3:-1]:
        try:
            img = urllib.request.urlretrieve(j, images_dir + '/' + ("+".join(X_train['concepts_processed'][i])) + "_"+str(count) + ".png")
            p.append(img[0])
            count += 1
        except:
            continue

    X_train.loc[i, 'Image_path'] = str(p)
    p1.append(p)
    print(count)

    # Create a checkpoint every checkpoint_interval iterations
    if i % checkpoint_interval == 0:
        with open('/content/checkpoint_' + str(i) + '.pkl', 'wb') as f:
            pickle.dump([X_train, p1, count,i], f)




In [23]:
X_test

,concepts,concept_set_idx,target,concepts_processed,concepts_new,Image_path
0,['basket' 'fruit' 'orange' 'include'],25371,Multiple baskets filled with fruit including o...,"[basket, fruit, orange, include]",basket+fruit+orange+include,['/content/drive/MyDrive/Retrieve_caption_gene...
1,['basket' 'fry' 'topping' 'hotdog'],27622,A hotdog with toppings next to a basket of sea...,"[basket, fry, topping, hotdog]",basket+fry+topping+hotdog,[]
2,['bassist' 'concert' 'live' 'perform'],27974,bassist of filter performs live in concert,"[bassist, concert, live, perform]",bassist+concert+live+perform,['/content/drive/MyDrive/Retrieve_caption_gene...
3,['bathroom' 'cabinet' 'window' 'sink'],25587,"A bathroom with a sink, counter, cabinets and ...","[bathroom, cabinet, window, sink]",bathroom+cabinet+window+sink,['/content/drive/MyDrive/Retrieve_caption_gene...
4,['bathroom' 'checker' 'sink' 'floor'],25400,a bathroom with a sink and a checkered floor,"[bathroom, checker, sink, floor]",bathroom+checker+sink+floor,['/content/drive/MyDrive/Retrieve_caption_gene...
...,...,...,...,...,...,...
195,['building' 'city' 'horizon' 'illuminate' 'road'],30846,asphalt road and city with illuminated buildin...,"[building, city, horizon, illuminate, road]",building+city+horizon+illuminate+road,['/content/drive/MyDrive/Retrieve_caption_gene...
196,['building' 'cloud' 'reflect' 'sky' 'facade'],29503,sky and clouds reflecting over the facades of ...,"[building, cloud, reflect, sky, facade]",building+cloud+reflect+sky+facade,['/content/drive/MyDrive/Retrieve_caption_gene...
197,['building' 'colour' 'detail' 'add' 'attention'],32376,attention to details and vibrant colours add t...,"[building, colour, detail, add, attention]",building+colour+detail+add+attention,['/content/drive/MyDrive/Retrieve_caption_gene...
198,['building' 'detail' 'attention' 'appeal' 'add'],32379,attention to details and vibrant colours add t...,"[building, detail, attention, appeal, add]",building+detail+attention+appeal+add,[]


In [24]:
X_train.to_csv('X_train_final_rd.csv')
X_test.to_csv('X_test_final_rd.csv')

Filtering out those concepts whose images could not be retrieved.

In [26]:
index=[]
for i in range(len(X_test)):
  if X_test['Image_path'][i]=='[]':
    index.append(i)
index

[1, 10, 39, 51, 53, 86, 91, 125, 141, 142, 174, 176, 185, 186, 198]

In [29]:
len(index)

15

In [ ]:
index1=[]
for i in range(len(X_train)):
  if X_train['Image_path'][i]=='[]':
    index1.append(i)
index1

In [40]:
train_df_dropped = X_train.drop(index1, axis=0)

In [27]:
test_df_dropped=X_test.drop(index,axis=0)

In [28]:
test_df_dropped

,concepts,concept_set_idx,target,concepts_processed,concepts_new,Image_path
0,['basket' 'fruit' 'orange' 'include'],25371,Multiple baskets filled with fruit including o...,"[basket, fruit, orange, include]",basket+fruit+orange+include,['/content/drive/MyDrive/Retrieve_caption_gene...
2,['bassist' 'concert' 'live' 'perform'],27974,bassist of filter performs live in concert,"[bassist, concert, live, perform]",bassist+concert+live+perform,['/content/drive/MyDrive/Retrieve_caption_gene...
3,['bathroom' 'cabinet' 'window' 'sink'],25587,"A bathroom with a sink, counter, cabinets and ...","[bathroom, cabinet, window, sink]",bathroom+cabinet+window+sink,['/content/drive/MyDrive/Retrieve_caption_gene...
4,['bathroom' 'checker' 'sink' 'floor'],25400,a bathroom with a sink and a checkered floor,"[bathroom, checker, sink, floor]",bathroom+checker+sink+floor,['/content/drive/MyDrive/Retrieve_caption_gene...
5,['bathroom' 'contain' 'toilet' 'wall'],25348,A bathroom containing a very dirty wall and to...,"[bathroom, contain, toilet, wall]",bathroom+contain+toilet+wall,['/content/drive/MyDrive/Retrieve_caption_gene...
...,...,...,...,...,...,...
194,['building' 'attention' 'add' 'appeal' 'renova...,32375,attention to details and vibrant colours add t...,"[building, attention, add, appeal, renovate]",building+attention+add+appeal+renovate,['/content/drive/MyDrive/Retrieve_caption_gene...
195,['building' 'city' 'horizon' 'illuminate' 'road'],30846,asphalt road and city with illuminated buildin...,"[building, city, horizon, illuminate, road]",building+city+horizon+illuminate+road,['/content/drive/MyDrive/Retrieve_caption_gene...
196,['building' 'cloud' 'reflect' 'sky' 'facade'],29503,sky and clouds reflecting over the facades of ...,"[building, cloud, reflect, sky, facade]",building+cloud+reflect+sky+facade,['/content/drive/MyDrive/Retrieve_caption_gene...
197,['building' 'colour' 'detail' 'add' 'attention'],32376,attention to details and vibrant colours add t...,"[building, colour, detail, add, attention]",building+colour+detail+add+attention,['/content/drive/MyDrive/Retrieve_caption_gene...


# Checking the distribution of the concepts in our final train and test sets

In [34]:
filtered_df = test_df_dropped[test_df_dropped['concepts_processed'].map(len)==3]

In [35]:
len(filtered_df)

0

In [36]:
type(X_test['Image_path'][0])

str

In [52]:
import ast
train_df_dropped['Image_path_new']=train_df_dropped['Image_path'].apply(lambda x: ast.literal_eval(x))
train_df_dropped['Image_path_new'][0]

['/content/drive/MyDrive/Retrieve_caption_generate/train_images/abacus+help+learn_0.png',
 '/content/drive/MyDrive/Retrieve_caption_generate/train_images/abacus+help+learn_1.png']

In [37]:
import ast
test_df_dropped['Image_path_new']=test_df_dropped['Image_path'].apply(lambda x: ast.literal_eval(x))
test_df_dropped['Image_path_new'][0]

['/content/drive/MyDrive/Retrieve_caption_generate/test_images/basket+fruit+orange+include_0.png']

# Caption!

I tried using two image captioning models - Vision Transformer( ViT)-GPT2 and Blip, observed that Blip is performing better on this dataset and hence used it for generating captions.

In [48]:
import torch
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from PIL import Image

# Load the pre-trained model
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Load the image processor
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Set the maximum length of the generated captions
max_length = 16

# Set the number of beams to use for beam search
num_beams = 4

# Define a function to generate captions for a list of image paths
def predict_step(image_paths):
    # Load the images
    images = []
    for image_path in image_paths:
        image = Image.open(image_path)
        if image.mode != "RGB":
            image = image.convert(mode="RGB")
        images.append(image)

    # Extract features from the images
    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    # Generate captions
    output_ids = model.generate(pixel_values, max_length=max_length, num_beams=num_beams)
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

# Generate captions for a single image
image_path = X_train['Image_path_new'][1][-1]
captions = predict_step([image_path])
print(captions)


(…)mage-captioning/resolve/main/config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

(…)ng/resolve/main/preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

(…)oning/resolve/main/tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

(…)image-captioning/resolve/main/vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

(…)image-captioning/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)e-captioning/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

(…)ing/resolve/main/special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


['a boat sitting on top of a body of water']


In [39]:
test_df_dropped

,concepts,concept_set_idx,target,concepts_processed,concepts_new,Image_path,Image_path_new
0,['basket' 'fruit' 'orange' 'include'],25371,Multiple baskets filled with fruit including o...,"[basket, fruit, orange, include]",basket+fruit+orange+include,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...
2,['bassist' 'concert' 'live' 'perform'],27974,bassist of filter performs live in concert,"[bassist, concert, live, perform]",bassist+concert+live+perform,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...
3,['bathroom' 'cabinet' 'window' 'sink'],25587,"A bathroom with a sink, counter, cabinets and ...","[bathroom, cabinet, window, sink]",bathroom+cabinet+window+sink,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...
4,['bathroom' 'checker' 'sink' 'floor'],25400,a bathroom with a sink and a checkered floor,"[bathroom, checker, sink, floor]",bathroom+checker+sink+floor,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...
5,['bathroom' 'contain' 'toilet' 'wall'],25348,A bathroom containing a very dirty wall and to...,"[bathroom, contain, toilet, wall]",bathroom+contain+toilet+wall,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...
...,...,...,...,...,...,...,...
194,['building' 'attention' 'add' 'appeal' 'renova...,32375,attention to details and vibrant colours add t...,"[building, attention, add, appeal, renovate]",building+attention+add+appeal+renovate,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...
195,['building' 'city' 'horizon' 'illuminate' 'road'],30846,asphalt road and city with illuminated buildin...,"[building, city, horizon, illuminate, road]",building+city+horizon+illuminate+road,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...
196,['building' 'cloud' 'reflect' 'sky' 'facade'],29503,sky and clouds reflecting over the facades of ...,"[building, cloud, reflect, sky, facade]",building+cloud+reflect+sky+facade,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...
197,['building' 'colour' 'detail' 'add' 'attention'],32376,attention to details and vibrant colours add t...,"[building, colour, detail, add, attention]",building+colour+detail+add+attention,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...


In [17]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")         #.to("cuda")
def generate_caption(l):
  try:
#img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
    raw_image = Image.open(l[0])


# unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt")            #.to("cuda")

    out = model.generate(**inputs)
    return(processor.decode(out[0], skip_special_tokens=True))
  except:
    raw_image = Image.open(l[1])


# unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt")            #.to("cuda")

    out = model.generate(**inputs)
    return(processor.decode(out[0], skip_special_tokens=True))
train_df_dropped['caption']=train_df_dropped['Image_path_new'].progress_apply(lambda x:generate_caption(x))

  0%|          | 0/822 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [40]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")
def generate_caption(l):
  try:
#img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
    raw_image = Image.open(l[0])


# unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt").to("cuda")

    out = model.generate(**inputs)
    return(processor.decode(out[0], skip_special_tokens=True))
  except:
    raw_image = Image.open(l[1])


# unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt").to("cuda")

    out = model.generate(**inputs)
    return(processor.decode(out[0], skip_special_tokens=True))
test_df_dropped['caption']=test_df_dropped['Image_path_new'].progress_apply(lambda x:generate_caption(x))

(…)se/resolve/main/preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

(…)-base/resolve/main/tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

(…)e-captioning-base/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)tioning-base/resolve/main/tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

(…)ase/resolve/main/special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

(…)captioning-base/resolve/main/config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

  0%|          | 0/185 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [41]:
test_df_dropped.to_csv('/content/drive/MyDrive/Retrieve_caption_generate/test_captioned_df.csv')

In [43]:
test_df_dropped

,concepts,concept_set_idx,target,concepts_processed,concepts_new,Image_path,Image_path_new,caption
0,['basket' 'fruit' 'orange' 'include'],25371,Multiple baskets filled with fruit including o...,"[basket, fruit, orange, include]",basket+fruit+orange+include,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a basket of fruit and grapes on a table
2,['bassist' 'concert' 'live' 'perform'],27974,bassist of filter performs live in concert,"[bassist, concert, live, perform]",bassist+concert+live+perform,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a man playing a guitar on a stage
3,['bathroom' 'cabinet' 'window' 'sink'],25587,"A bathroom with a sink, counter, cabinets and ...","[bathroom, cabinet, window, sink]",bathroom+cabinet+window+sink,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a bathroom with marble floors and white cabinets
4,['bathroom' 'checker' 'sink' 'floor'],25400,a bathroom with a sink and a checkered floor,"[bathroom, checker, sink, floor]",bathroom+checker+sink+floor,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a bathroom with a toilet and sink in a house -...
5,['bathroom' 'contain' 'toilet' 'wall'],25348,A bathroom containing a very dirty wall and to...,"[bathroom, contain, toilet, wall]",bathroom+contain+toilet+wall,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a bathroom with black and white tiles and a to...
...,...,...,...,...,...,...,...,...
194,['building' 'attention' 'add' 'appeal' 'renova...,32375,attention to details and vibrant colours add t...,"[building, attention, add, appeal, renovate]",building+attention+add+appeal+renovate,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a house with flowers in front of it
195,['building' 'city' 'horizon' 'illuminate' 'road'],30846,asphalt road and city with illuminated buildin...,"[building, city, horizon, illuminate, road]",building+city+horizon+illuminate+road,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a city skyline at night with long exposure
196,['building' 'cloud' 'reflect' 'sky' 'facade'],29503,sky and clouds reflecting over the facades of ...,"[building, cloud, reflect, sky, facade]",building+cloud+reflect+sky+facade,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a modern office building with glass windows an...
197,['building' 'colour' 'detail' 'add' 'attention'],32376,attention to details and vibrant colours add t...,"[building, colour, detail, add, attention]",building+colour+detail+add+attention,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a painting of a building with stairs and stairs


# Generate!

In [53]:
import pandas as pd
X_train=pd.read_csv('/content/drive/MyDrive/Retrieve_caption_generate/train_captioned_df.csv')
X_train

,Unnamed: 0.1,Unnamed: 0,concepts,concept_set_idx,target,concepts_processed,concepts_new,Image_path,Image_path_new,caption
0,0,0,['abacus' 'help' 'learn'],20011,these young girls learn the art of calculating...,"['abacus', 'help', 'learn']",abacus+help+learn,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a hand holding an aban with a pencil
1,1,1,['abandon' 'bank' 'boat'],3747,an abandoned boat lies on the bank .,"['abandon', 'bank', 'boat']",abandon+bank+boat,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,old rusty boat stock photo image of rusty
2,2,2,['abandon' 'building' 'prairie'],14195,windmill and buildings of an old abandoned hom...,"['abandon', 'building', 'prairie']",abandon+building+prairie,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,an old stone barn in a field under a stormy sk...
3,3,3,['abandon' 'building' 'town'],3509,a derelict building in the abandoned town,"['abandon', 'building', 'town']",abandon+building+town,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a staircase in an old building with a broken wall
4,4,4,['abandon' 'chair' 'asylum'],16115,chairs in an abandoned insane asylum,"['abandon', 'chair', 'asylum']",abandon+chair+asylum,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a room with a chair and a sink
...,...,...,...,...,...,...,...,...,...,...
817,895,895,['block' 'flat' 'sky' 'post' 'corner'],31802,a block of flats viewed up towards the patchy ...,"['block', 'flat', 'sky', 'post', 'corner']",block+flat+sky+post+corner,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,"the golf club, sector 1, sector 2, sector 3, s..."
818,896,896,['block' 'flower' 'rear' 'grow' 'garden'],31463,macro image of flowers growing in the ornament...,"['block', 'flower', 'rear', 'grow', 'garden']",block+flower+rear+grow+garden,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a garden with lots of flowers and plants
819,897,897,['block' 'sun' 'frond' 'tree' 'cloud'],31830,tropical storm clouds block out the sun as win...,"['block', 'sun', 'frond', 'tree', 'cloud']",block+sun+frond+tree+cloud,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,palm tree with christmas lights
820,898,898,['block' 'tree' 'cloud' 'wind' 'sun'],31834,tropical storm clouds block out the sun as win...,"['block', 'tree', 'cloud', 'wind', 'sun']",block+tree+cloud+wind+sun,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a white shade over a beach with palm trees


In [54]:
X_train=X_train.drop(['Unnamed: 0.1', 'Unnamed: 0'],axis=1)
X_train

,concepts,concept_set_idx,target,concepts_processed,concepts_new,Image_path,Image_path_new,caption
0,['abacus' 'help' 'learn'],20011,these young girls learn the art of calculating...,"['abacus', 'help', 'learn']",abacus+help+learn,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a hand holding an aban with a pencil
1,['abandon' 'bank' 'boat'],3747,an abandoned boat lies on the bank .,"['abandon', 'bank', 'boat']",abandon+bank+boat,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,old rusty boat stock photo image of rusty
2,['abandon' 'building' 'prairie'],14195,windmill and buildings of an old abandoned hom...,"['abandon', 'building', 'prairie']",abandon+building+prairie,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,an old stone barn in a field under a stormy sk...
3,['abandon' 'building' 'town'],3509,a derelict building in the abandoned town,"['abandon', 'building', 'town']",abandon+building+town,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a staircase in an old building with a broken wall
4,['abandon' 'chair' 'asylum'],16115,chairs in an abandoned insane asylum,"['abandon', 'chair', 'asylum']",abandon+chair+asylum,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a room with a chair and a sink
...,...,...,...,...,...,...,...,...
817,['block' 'flat' 'sky' 'post' 'corner'],31802,a block of flats viewed up towards the patchy ...,"['block', 'flat', 'sky', 'post', 'corner']",block+flat+sky+post+corner,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,"the golf club, sector 1, sector 2, sector 3, s..."
818,['block' 'flower' 'rear' 'grow' 'garden'],31463,macro image of flowers growing in the ornament...,"['block', 'flower', 'rear', 'grow', 'garden']",block+flower+rear+grow+garden,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a garden with lots of flowers and plants
819,['block' 'sun' 'frond' 'tree' 'cloud'],31830,tropical storm clouds block out the sun as win...,"['block', 'sun', 'frond', 'tree', 'cloud']",block+sun+frond+tree+cloud,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,palm tree with christmas lights
820,['block' 'tree' 'cloud' 'wind' 'sun'],31834,tropical storm clouds block out the sun as win...,"['block', 'tree', 'cloud', 'wind', 'sun']",block+tree+cloud+wind+sun,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a white shade over a beach with palm trees


In [55]:
X_train['caption'][1]

'old rusty boat stock photo image of rusty'

In [56]:
train_df_dropped = X_train.drop(index, axis=0)
train_df_dropped

,concepts,concept_set_idx,target,concepts_processed,concepts_new,Image_path,Image_path_new,caption
0,['abacus' 'help' 'learn'],20011,these young girls learn the art of calculating...,"['abacus', 'help', 'learn']",abacus+help+learn,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a hand holding an aban with a pencil
2,['abandon' 'building' 'prairie'],14195,windmill and buildings of an old abandoned hom...,"['abandon', 'building', 'prairie']",abandon+building+prairie,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,an old stone barn in a field under a stormy sk...
3,['abandon' 'building' 'town'],3509,a derelict building in the abandoned town,"['abandon', 'building', 'town']",abandon+building+town,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a staircase in an old building with a broken wall
4,['abandon' 'chair' 'asylum'],16115,chairs in an abandoned insane asylum,"['abandon', 'chair', 'asylum']",abandon+chair+asylum,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a room with a chair and a sink
5,['abandon' 'countryside' 'building'],15711,abandoned building in the rural countryside,"['abandon', 'countryside', 'building']",abandon+countryside+building,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,an old house in the countryside of ukraine
...,...,...,...,...,...,...,...,...
817,['block' 'flat' 'sky' 'post' 'corner'],31802,a block of flats viewed up towards the patchy ...,"['block', 'flat', 'sky', 'post', 'corner']",block+flat+sky+post+corner,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,"the golf club, sector 1, sector 2, sector 3, s..."
818,['block' 'flower' 'rear' 'grow' 'garden'],31463,macro image of flowers growing in the ornament...,"['block', 'flower', 'rear', 'grow', 'garden']",block+flower+rear+grow+garden,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a garden with lots of flowers and plants
819,['block' 'sun' 'frond' 'tree' 'cloud'],31830,tropical storm clouds block out the sun as win...,"['block', 'sun', 'frond', 'tree', 'cloud']",block+sun+frond+tree+cloud,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,palm tree with christmas lights
820,['block' 'tree' 'cloud' 'wind' 'sun'],31834,tropical storm clouds block out the sun as win...,"['block', 'tree', 'cloud', 'wind', 'sun']",block+tree+cloud+wind+sun,['/content/drive/MyDrive/Retrieve_caption_gene...,['/content/drive/MyDrive/Retrieve_caption_gene...,a white shade over a beach with palm trees


In [57]:
import ast
train_df_dropped['Image_path_new']=train_df_dropped['Image_path'].apply(lambda x: ast.literal_eval(x))
train_df_dropped['Image_path_new'][0]

['/content/drive/MyDrive/Retrieve_caption_generate/train_images/abacus+help+learn_0.png',
 '/content/drive/MyDrive/Retrieve_caption_generate/train_images/abacus+help+learn_1.png']

## Training the T5-base model on the training data without using captions to see how the model performs without the aid of visual captions.

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import pickle
from IPython.display import HTML, display
batch_size=8
num_of_batches=int(len(X_train)/batch_size)
loss_per_10_steps=[]

# Define the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)

# Move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set the model in training mode
model.train()

# Define the optimizer
optimizer = Adafactor(model.parameters(), lr=1e-3, eps=(1e-30, 1e-3), clip_threshold=1.0, decay_rate=-0.8, beta1=None, weight_decay=0.0, relative_step=False, scale_parameter=False, warmup_init=False)

# Define the progress bar function
def progress(loss, value, max=100):
    return HTML(""" Batch loss :{loss}
    <progress
value='{value}'max='{max}',style='width: 100%'>{value}
    </progress>
    """.format(loss=loss, value=value, max=max))

# Train the model for 15 epochs
for epoch in range(1, 25):
    print('Running epoch: {}'.format(epoch))

    running_loss = 0

    out = display(progress(1, len(X_train) // batch_size + 1), display_id=True)
    for i in range(len(X_train) // batch_size):
        input_batch = []
        label_batch = []

        new_df = X_train[i * batch_size:(i + 1) * batch_size]
        for indx, row in new_df.iterrows():
            input =  " ".join(row['concepts_processed'])+'</s>'
            labels = row['target'] + '</s>'
            input_batch.append(input)
            label_batch.append(labels)

        input_batch = tokenizer.batch_encode_plus(input_batch, padding=True, max_length=400, return_tensors='pt')["input_ids"]
        label_batch = tokenizer.batch_encode_plus(label_batch, padding=True, max_length=400, return_tensors="pt")["input_ids"]

        # Move the input and target batches to the GPU
        input_batch = input_batch.to(device)
        label_batch = label_batch.to(device)

        # Clear out the gradients of all Variables
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=input_batch, labels=label_batch)
        loss = outputs.loss
        loss_num = loss.item()
        logits = outputs.logits
        running_loss += loss_num

        if i % 10 == 0:
            loss_per_10_steps.append(loss_num)
            out.update(progress(loss_num, i, len(X_train) // batch_size + 1))

        # Calculating the gradients
        loss.backward()

        # Updating the params
        optimizer.step()

    running_loss = running_loss / int(len(X_train) // batch_size)
    print('Epoch: {} , Running loss: {}'.format(epoch, running_loss))


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Running epoch: 1


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch: 1 , Running loss: 2.878064931607714
Running epoch: 2


Epoch: 2 , Running loss: 1.992506166299184
Running epoch: 3


...

Epoch: 20 , Running loss: 0.1242996681408555
Running epoch: 21


Epoch: 21 , Running loss: 0.10766916544925348
Running epoch: 22


Epoch: 22 , Running loss: 0.1032779105372873
Running epoch: 23


Epoch: 23 , Running loss: 0.09320204536996636
Running epoch: 24


Epoch: 24 , Running loss: 0.09463708572016627


In [ ]:
torch.save(model.state_dict(),'train_base_model.bin')
model.load_state_dict(torch.load('/content/train_base_model.bin'))
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def generate(row,model=model,tokenizer=tokenizer):
  l=[]
  total_bleu = 0
  predictions = []
  targets = []
  model.eval()
  input_ids = tokenizer.encode(" ".join(row['concepts_processed']) +'</s>',
                               return_tensors="pt")
  input_ids = input_ids.to(model.device)
  outputs = model.generate(input_ids)
  l.append(tokenizer.decode(outputs[0]))
  return(l)
  #predictions.append(generated_text)
  #targets.append(text['target'][i])

    # Calculate BLEU score
  #bleu_score = corpus_bleu(predictions, [targets], smooth_method='none')
  #total_bleu += bleu_score.score

  #return total_bleu / len(test_data)

# Evaluate the model on the test set
test_df_dropped['predicted_base']=test_df_dropped.progress_apply(generate,axis=1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/185 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
import nltk
hypothesis=test_df_dropped['predicted_base'].tolist()
reference=test_df_dropped['target'].tolist()
def bleu(hypo,ref):
  bleu_sc=0
  for i in range(len(reference)):
    h=hypo[i][0].strip('<pad></s>')
    h=h.lstrip(' ')
    h_list=h.split(" ")
    ref_list=ref[i].split(" ")
    bleuscore=nltk.translate.bleu_score.sentence_bleu([ref_list],h_list)
    bleu_sc+=bleuscore
  return(bleu_sc/len(reference))
bleu(hypothesis,reference)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

0.00573601478371524

## Using T5 Base for generating captions. The pre-trained T5-base model is finetuned on our training data + image captions with the optimal hyperparameters as mentioned in the paper and used for evaluation on our test dataset.

In [86]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import pickle
from IPython.display import HTML, display
batch_size=8
num_of_batches=int(len(X_train)/batch_size)
loss_per_10_steps=[]

# Define the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)

# Move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set the model in training mode
model.train()

# Define the optimizer
optimizer = Adafactor(model.parameters(), lr=1e-3, eps=(1e-30, 1e-3), clip_threshold=1.0, decay_rate=-0.8, beta1=None, weight_decay=0.0, relative_step=False, scale_parameter=False, warmup_init=False)

# Define the progress bar function
def progress(loss, value, max=100):
    return HTML(""" Batch loss :{loss}
    <progress
value='{value}'max='{max}',style='width: 100%'>{value}
    </progress>
    """.format(loss=loss, value=value, max=max))

# Train the model for 15 epochs
for epoch in range(1, 25):
    print('Running epoch: {}'.format(epoch))

    running_loss = 0

    out = display(progress(1, len(X_train) // batch_size + 1), display_id=True)
    for i in range(len(X_train) // batch_size):
        input_batch = []
        label_batch = []

        new_df = X_train[i * batch_size:(i + 1) * batch_size]
        for indx, row in new_df.iterrows():
            input =  " ".join(row['concepts_processed']) +' <s> '+ row['caption'] +'</s>'
            labels = row['target'] + '</s>'
            input_batch.append(input)
            label_batch.append(labels)

        input_batch = tokenizer.batch_encode_plus(input_batch, padding=True, max_length=400, return_tensors='pt')["input_ids"]
        label_batch = tokenizer.batch_encode_plus(label_batch, padding=True, max_length=400, return_tensors="pt")["input_ids"]

        # Move the input and target batches to the GPU
        input_batch = input_batch.to(device)
        label_batch = label_batch.to(device)

        # Clear out the gradients of all Variables
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=input_batch, labels=label_batch)
        loss = outputs.loss
        loss_num = loss.item()
        logits = outputs.logits
        running_loss += loss_num

        if i % 10 == 0:
            loss_per_10_steps.append(loss_num)
            out.update(progress(loss_num, i, len(X_train) // batch_size + 1))

        # Calculating the gradients
        loss.backward()

        # Updating the params
        optimizer.step()

    running_loss = running_loss / int(len(X_train) // batch_size)
    print('Epoch: {} , Running loss: {}'.format(epoch, running_loss))


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Running epoch: 1


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch: 1 , Running loss: 2.625473996003469
Running epoch: 2


Epoch: 2 , Running loss: 1.7912203465022294
Running epoch: 3


...

Epoch: 20 , Running loss: 0.07370101178393644
Running epoch: 21


Epoch: 21 , Running loss: 0.062290248602592184
Running epoch: 22


Epoch: 22 , Running loss: 0.06438735529652559
Running epoch: 23


Epoch: 23 , Running loss: 0.061572485626218657
Running epoch: 24


Epoch: 24 , Running loss: 0.051694339989255385


In [46]:
test_df_dropped

,concepts,concept_set_idx,target,concepts_processed,concepts_new,Image_path,Image_path_new,caption
0,['basket' 'fruit' 'orange' 'include'],25371,Multiple baskets filled with fruit including o...,"[basket, fruit, orange, include]",basket+fruit+orange+include,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a basket of fruit and grapes on a table
2,['bassist' 'concert' 'live' 'perform'],27974,bassist of filter performs live in concert,"[bassist, concert, live, perform]",bassist+concert+live+perform,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a man playing a guitar on a stage
3,['bathroom' 'cabinet' 'window' 'sink'],25587,"A bathroom with a sink, counter, cabinets and ...","[bathroom, cabinet, window, sink]",bathroom+cabinet+window+sink,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a bathroom with marble floors and white cabinets
4,['bathroom' 'checker' 'sink' 'floor'],25400,a bathroom with a sink and a checkered floor,"[bathroom, checker, sink, floor]",bathroom+checker+sink+floor,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a bathroom with a toilet and sink in a house -...
5,['bathroom' 'contain' 'toilet' 'wall'],25348,A bathroom containing a very dirty wall and to...,"[bathroom, contain, toilet, wall]",bathroom+contain+toilet+wall,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a bathroom with black and white tiles and a to...
...,...,...,...,...,...,...,...,...
194,['building' 'attention' 'add' 'appeal' 'renova...,32375,attention to details and vibrant colours add t...,"[building, attention, add, appeal, renovate]",building+attention+add+appeal+renovate,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a house with flowers in front of it
195,['building' 'city' 'horizon' 'illuminate' 'road'],30846,asphalt road and city with illuminated buildin...,"[building, city, horizon, illuminate, road]",building+city+horizon+illuminate+road,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a city skyline at night with long exposure
196,['building' 'cloud' 'reflect' 'sky' 'facade'],29503,sky and clouds reflecting over the facades of ...,"[building, cloud, reflect, sky, facade]",building+cloud+reflect+sky+facade,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a modern office building with glass windows an...
197,['building' 'colour' 'detail' 'add' 'attention'],32376,attention to details and vibrant colours add t...,"[building, colour, detail, add, attention]",building+colour+detail+add+attention,['/content/drive/MyDrive/Retrieve_caption_gene...,[/content/drive/MyDrive/Retrieve_caption_gener...,a painting of a building with stairs and stairs


In [87]:
torch.save(model.state_dict(),'train_model.bin')
model.load_state_dict(torch.load('/content/train_model.bin'))
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def generate(row,model=model,tokenizer=tokenizer):
  l=[]
  total_bleu = 0
  predictions = []
  targets = []
  model.eval()
  input_ids = tokenizer.encode(" ".join(row['concepts_processed']) +' <s> '+ row['caption'] +'</s>',
                               return_tensors="pt")
  input_ids = input_ids.to(model.device)
  outputs = model.generate(input_ids)
  l.append(tokenizer.decode(outputs[0]))
  return(l)
  #predictions.append(generated_text)
  #targets.append(text['target'][i])

    # Calculate BLEU score
  #bleu_score = corpus_bleu(predictions, [targets], smooth_method='none')
  #total_bleu += bleu_score.score

  #return total_bleu / len(test_data)

# Evaluate the model on the test set
test_df_dropped['predicted']=test_df_dropped.progress_apply(generate,axis=1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/185 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [61]:
test_df_dropped['predicted'][0]

['<pad> A wire basket with apples and oranges in it.</s>']

In [62]:
test_df_dropped['target'][0]

'Multiple baskets filled with fruit including oranges and pears.'

In [88]:
import nltk
hypothesis=test_df_dropped['predicted'].tolist()
reference=test_df_dropped['target'].tolist()
def bleu(hypo,ref):
  bleu_sc=0
  for i in range(len(reference)):
    h=hypo[i][0].strip('<pad></s>')
    h=h.lstrip(' ')
    h_list=h.split(" ")
    ref_list=ref[i].split(" ")
    bleuscore=nltk.translate.bleu_score.sentence_bleu([ref_list],h_list)
    bleu_sc+=bleuscore
  return(bleu_sc/len(reference))
bleu(hypothesis,reference)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

0.02370649845436812

# Conclusion:
## We observe that the Bleu score, which is a measure of similarity between a hypothesized sentence generated by a model and a reference sentence, is around **0.005** for the model without captions and it is **0.02** for the model with captions (VisCTG). **The Bleu score is almost 4 times higher with visual grounding!** herefore, it can be clearly inferred that visual grounding improves the performace of models in NLG task.